# USE ENV si_env_rolling

In [1]:
# import packages
%matplotlib qt
import glob as glob
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
from probeinterface import Probe, ProbeGroup, generate_tetrode
import json as json
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

# for path
animal = 'Bayleef'
folder_name = 'Bayleef_2025-01-30_11-53-57_010'

# import data
data_path = fr'L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\{folder_name}'

recording = se.read_openephys(data_path)
events = se.read_openephys_event(data_path)
print(recording)
# with open(glob.glob(data_path+'/structure.oebin', recursive = True)[0], 'r') as f:
#         oebin = json.loads(f.read())

# select ephys channels only
selection = ['CH'+str(i) for i in np.arange(1,65)]
ephys_recording = recording.select_channels(selection)

# make a channel map 
channelmapoe = [40, 38, 36, 34,
                48, 46, 44, 42,
                56, 54, 52, 50,
                58, 64, 62, 60,
                63, 61, 59, 57,
                55, 53, 51, 49,
                47, 45, 43, 41,
                39, 37, 35, 33,
                25, 27, 29, 31,
                17, 19, 21, 23,
                9, 11, 13, 15,
                1, 3, 5, 7,
                4, 6, 8, 2,
                10, 12, 14, 16,
                18, 20, 22, 24,
                26, 28, 30, 32]

# subtract 1 to fit python indexing
channelmappythonic = np.array(channelmapoe) - 1

# create probegroup and set channel locations for sorting purposes
probegroup = ProbeGroup()
for i in range(16): # we have 16 tetrodes
    tetrode = generate_tetrode()
    tetrode.move([i * 300, 0]) # make lots of space between tetrodes
    probegroup.add_probe(tetrode)

probegroup.set_global_device_channel_indices(channelmappythonic) # tetrodes arranged based on channelmap
ephys_recording.set_probegroup(probegroup, group_mode='by_probe', in_place=True)


# preprocess (havent yet ditched disconnected channels from this!)
# recording_f = spre.bandpass_filter(ephys_recording, freq_min=300, freq_max=6000)
# recording_cmr = spre.common_reference(recording_f, reference='global', operator='median') 

OpenEphysBinaryRecordingExtractor: 75 channels - 30.0kHz - 1 segments - 95,414,526 samples 
                                   3,180.48s (53.01 minutes) - int16 dtype - 13.33 GiB


ChannelSliceRecording: 64 channels - 30.0kHz - 1 segments - 95,414,526 samples 
                       3,180.48s (53.01 minutes) - int16 dtype - 11.37 GiB

In [ ]:
# sorting 
# sorted_path = 
aggregate_sorting = ss.run_sorter_by_property(
    sorter_name = 'mountainsort5',
    recording = recording_cmr,
    grouping_property = "group",
    folder = "sort_by_group", engine = "joblib", 
    engine_kwargs = {"n_jobs": 4}
)

OpenEphysBinaryRecordingExtractor: 75 channels - 30.0kHz - 1 segments - 114,966,671 samples 
                                   3,832.22s (1.06 hours) - int16 dtype - 16.06 GiB


ValueError: Folder C:\Users\dlab\rishika_sim\sort_by_group\3 already exists

In [48]:
# play video with filename
import cv2
import numpy as np

expt = '4portProb_ephys'
box = 'Box1_ephys'
animal = 'Dratini'

# will work using videos 
vid_fname = 'Dratini_20241012_sess_1' 
cap = cv2.VideoCapture(f'L:/{expt}/{box}/{animal}/Video/{vid_fname}.mp4')

frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

myFrameNumber = 294300

# check for valid frame number
if myFrameNumber >= 0 & myFrameNumber <= frameCount:
    # set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, myFrameNumber)

#Read the frame from the video. 
ret, frame = cap.read()
assert ret, "We failed to read the first frame!"

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # flip the colors. opencv for some dumb reason uses bgr

# Change the cropping to cut only the LEDs - might be specific to a video - remove as much background as possible!
ledx_sync = [20, 35]
ledy_sync = [260, 280]
ledx_gpio = [560, 600]
ledy_gpio = [240, 270]

# plot the cropped leds
fig, axes = plt.subplots(1, 3, figsize=(10,5))

axes[0].imshow(gray)
gpio_rect = plt.Rectangle(xy=[ledx_gpio[0], ledy_gpio[0]],
                          width=np.diff(ledx_gpio)[0],
                          height=np.diff(ledy_gpio)[0],
                          facecolor='none', edgecolor='white')
sync_rect = plt.Rectangle(xy=[ledx_sync[0], ledy_sync[0]],
                          width=np.diff(ledx_sync)[0],
                          height=np.diff(ledy_sync)[0],
                          facecolor='none', edgecolor='white')

axes[0].add_artist(gpio_rect)
axes[0].add_artist(sync_rect)

axes[1].imshow(gray[ledy_sync[0]:ledy_sync[1], ledx_sync[0]:ledx_sync[1]])

axes[2].imshow(gray[ledy_gpio[0]:ledy_gpio[1], ledx_gpio[0]:ledx_gpio[1]])

In [40]:
# get the brightness trace for each led 
brightness_gpio = np.zeros(frameCount)  # that's the number of frames
brightness_sync = np.zeros(frameCount)

success = True
frame_id = 0
# load video again in case the fram enumber set is wrong
cap = cv2.VideoCapture(f'L:/{expt}/{box}/{animal}/Video/{vid_fname}.mp4')
# cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
while success:
    success, frame = cap.read()
    if success:
        r_chn_frame = frame[:, :, 0]
        brightness_gpio[frame_id] = r_chn_frame[ledy_gpio[0]:ledy_gpio[1], ledx_gpio[0]:ledx_gpio[1]].mean()
        brightness_sync[frame_id] = r_chn_frame[ledy_sync[0]:ledy_sync[1], ledx_sync[0]:ledx_sync[1]].mean()
        frame_id += 1
cap.release()

In [47]:
%matplotlib qt
plt.figure(figsize=(10,3))
plt.plot(brightness_gpio)
plt.plot(brightness_sync)
plt.axhline(110, color ='k')

In [ ]:
# add a threshold to detect events 
threshold_gpio = ...
threshold_sync = ...
brightness_gpio_trace = (brightness_gpio>threshold_gpio).astype(int)
brightness_sync_trace = (brightness_sync>threshold_sync).astype(int)

def frame2oe(frame_index):
  """Return the openephys sample corresponding to frame onset

  Args:
    frame_index (int): index of the frame in the avi movie

  Return:
    onset (int): index of the first sample of the frame in the OE recording
    offset (int): index of the last sample of the frame in the OE recording
  """
  # find the correct sync pulse:
  pulse_index = frame_sync_pulse_index[frame_index]
  # and where that pulse was in the recording
  oe_on = frame_start_index[pulse_index]
  oe_off = frame_stop_index[pulse_index]
  return oe_on, oe_off

# Sync OpCon events with OE

In [ ]:
# this takes time to load so only do it if reqd. check if all samples exist
acq_samples = np.load(fr'L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\{folder_name}\Record Node 101\experiment1\recording1\continuous\OE_FPGA_Acquisition_Board-100.Rhythm Data\sample_numbers.npy')

# generate the clock pulse for oe 
pulse = np.ones(acq_samples.shape)
for i in range(0, acq_samples.shape[0], 60000):
    pulse[i:i + 30000] = -1

# when did camera recording start?
# ??? align using session events for this case?
# start sync LED only when camera is already on, idiot
# aligning events from ephys to video?
events_data_path = fr'L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\{folder_name}\Record Node 101\experiment1\recording1\events\OE_FPGA_Acquisition_Board-100.Rhythm Data\TTL\*'
events = np.load(glob.glob(events_data_path+'full_words.npy', recursive = True)[0])
states = np.load(glob.glob(events_data_path+'states.npy', recursive = True)[0])
timestamps = np.load(glob.glob(events_data_path+'timestamps.npy', recursive = True)[0])
sample_numbers = np.load(glob.glob(events_data_path+'sample_numbers.npy', recursive = True)[0])
state_names = ['np1', 'np2', 'np3', 'np4', 'lick', 'tone', 'water']
event_names = np.array([state_names[i] for i in np.abs(states)-1])

# read opcon file for that day
# opcon_filepath = r"L:\4portProb_ephys\Box1_ephys\Dratini\Behavior\Dratini-2024-10-20-16-44-27.dat"
# names = ['event', 'value', 'sm', 'tp', 'smtime', 'eptime', 'dw', 'ww']
# import pandas as pd
# opcon_df = pd.read_csv(opcon_filepath, header = None, names = names)

# from opconNosepokeFunctions import *
# sessdf = trializer_v3(opcon_df, [13], 81, 51, 86, 61, 88, 83, 'Dratini') # just to see exactly how many trials there were

# np_mask = np.isin(states, [-1, -2, -3, -4]) # is this 1, 2, 3, 4?
# np_df = opcon_df[opcon_df.event.isin([21, 20, 19, 18]) & (opcon_df.value == 1) & (opcon_df.sm == 13) & (opcon_df.eptime > 1729508399)]
# np_df.loc[:, 'oe_times'] = np.nan
# np_df.loc[:, 'oe_events'] = np.nan

# np_df.iloc[:len(sample_numbers[np_mask]), -2] = sample_numbers[np_mask]
# np_df.iloc[:len(states[np_mask]), -1] = states[np_mask]
# np_df.loc[:, 'opcon_event'] = np_df['event'].replace({21:1, 20:2, 19:3, 18:4})

In [115]:
np_df = opcon_df[opcon_df.event.isin([21, 20, 19, 18]) & (opcon_df.value == 1) & (opcon_df.sm == 13) & (opcon_df.eptime > 1729508399)]


# Check for frame drops in video

In [42]:
# load camera timestamps to check if time difference is about the same and there are no frame drops
import pandas as pd
camera_timestamps = pd.read_csv(fr'L:\4portProb_ephys\Box1_ephys\Dratini\Video\{vid_fname}_timestamps.pts')
plt.plot(camera_timestamps.diff())
camera_timestamps.diff().value_counts()

# timecode format v2
11.108                  42097
11.105                  35070
11.107                  26352
11.106                  26186
11.107                  13244
                        ...  
11.105                      1
11.105                      1
11.105                      1
11.106                      1
22.217                      1
Name: count, Length: 766, dtype: int64

In [46]:
camera_timestamps.diff().value_counts()
plt.plot(camera_timestamps.diff())